In [1]:
import torch as t
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.models as models
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import Dataset, DataLoader, random_split
from datasets import load_dataset 
from PIL import Image
import os, random
import pandas as pd
import transformers as tr
import json
import shutil

c:\Users\nicco\OneDrive\Desktop\DHDK\2ndYear\courses\MLfH\ex5\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Repo card metadata block was not found. Setting CardData to empty.
Generating test split: 100%|██████████| 3000/3000 [00:00<00:00, 120378.39 examples/s]


In [2]:
lines = []
with open("hateful_memes/train.jsonl") as f:
    lines = f.read().splitlines()

lines_dict = [json.loads(line) for line in lines]

df = pd.DataFrame(lines_dict)
print(df.loc[[0]])

      id            img  label  \
0  42953  img/42953.png      0   

                                               text  
0  its their character not their color that matters  


In [12]:
# DO NOT RUN CELL!
'''
imgs_referenced = df["img"]
imgs_referenced.astype(str)

def select_ref_imgs(orig_loc):
    orig_loc = str(orig_loc)
    file_name = orig_loc[4:]
   
    try:
        shutil.move(f".\hateful_memes\{orig_loc}", f".\hateful_memes\imgs_present/{file_name}")
        print(f"file {file_name} moved")
    except FileNotFoundError:
        print(f"file {file_name} not found")
        pass
    except Exception as e:
        print(f"error:{e}")

def change_slash(string):
    string = string.replace("/",  "\ " )
    string = string.replace(" ", "")
    return string
    '''

# imgs_referenced = imgs_referenced.apply(change_slash)
# imgs_referenced.apply(select_ref_imgs)

print("files moved")

files moved


In [13]:
# DO NOT RUN CELL!

# for file_name in os.listdir("./hateful_memes/img"):
    # file_path = os.path.join("./hateful_memes/img", file_name)
    # os.remove(file_path)
    # print(f"{file_path} deleted.")
    # pass
print ("files deleted")

files deleted


In [ ]:
# DO NOT RUN CELL!

# shutil.rmtree("hateful_memes/img_old")

In [7]:
# DO NOT RUN CELL!

# os.rename("hateful_memes/imgs_present", "hateful_memes/img")

In [3]:
def check_imgs_exist(file_path):
    file_path = str(file_path)
    file_name = file_path[4:]
    if os.path.isfile(f"hateful_memes/{file_path}"):
        print(f"{file_name} present")
        return True
    else:
        return False
    
reference_df = df
reference_df["img_exists"] = reference_df["img"].apply(check_imgs_exist)
reference_df = reference_df[reference_df["img_exists"]]
reference_df = reference_df.drop(columns=["img_exists"])
reference_df = reference_df.reset_index()

42953.png present
13894.png present
82403.png present
16952.png present
76932.png present
70914.png present
02973.png present
58306.png present
79351.png present
34096.png present
19324.png present
79346.png present
13426.png present
28936.png present
59784.png present
02146.png present
70146.png present
28061.png present
97180.png present
49752.png present
25610.png present
15872.png present
72640.png present
17956.png present
93547.png present
10743.png present
37091.png present
76825.png present
51846.png present
95812.png present
53976.png present
70193.png present
60427.png present
78395.png present
04876.png present
92075.png present
86354.png present
74386.png present
59613.png present
10263.png present
52610.png present
58371.png present
27401.png present
89574.png present
25473.png present
47913.png present
08295.png present
02497.png present
90471.png present
69357.png present
54602.png present
15320.png present
63182.png present
82457.png present
42137.png present
74530.png 

In [4]:
print(reference_df.head(), len(reference_df))

   index     id            img  label  \
0      0  42953  img/42953.png      0   
1      2  13894  img/13894.png      0   
2      4  82403  img/82403.png      0   
3      5  16952  img/16952.png      0   
4      6  76932  img/76932.png      0   

                                                text  
0   its their character not their color that matters  
1                           putting bows on your pet  
2  everybody loves chocolate chip cookies, even h...  
3           go sports! do the thing! win the points!  
4     fine you're right. now can we fucking drop it?   6744


In [10]:
print(reference_df.query("id == '14507'"))

     index     id            img  label  \
882   1112  14507  img/14507.png      1   

                                                  text  
882  when your history teacher starts talking about...  


In [11]:
reference_df = reference_df.astype({"index":int, "id":int, "img":str, "label":float, "text":str})
data = reference_df

In [ ]:
class DatasetTxt (Dataset):
    def __init__(self, df, tokenizer):
        super().__init__()
        self.df = df
        self.tokenizer = tokenizer

        self.text_df = df[["text","label"]].astype({"text":str, "label":float})

    def __len__(self):
        return len(self.df["label"])
    
    def __getitem__(self, index):
        text = self.text_df["text"][index]
        label = self.text_df["label"][index]

        encoding = self.tokenizer.encode_plus(text, add_special_tokens=True, return_token_type_ids=False,
            padding='longest', return_attention_mask=True, return_tensors='pt', truncation=True)

        return {"input_ids": encoding["input_ids"].flatten(), "attention_mask": encoding["attention_mask"].flatten(),
                "labels": t.tensor(label, dtype=t.long)}
        
class DatasetTxt (Dataset):
    def __init__(self, df, img_dir):
        super().__init__()
        self.df = df
        self.img_dir = img_dir
        self.transform = transforms.Compose([
                    transforms.Resize((224, 224)),
                    transforms.ToTensor(),
                    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
                    ])

        self.image_df = df[["img","label"]]
        self.image_df = self.image_df.astype({"img":str, "label":float})

    def __len__(self):
        return len(self.df["label"])
    
    def __getitem__(self, index):
        img_pth = os.path.join(self.img_dir, self.image_df["img"])
        img = Image.open(img_pth).convert("RGB")
        img_tensor = self.transform(img).unsqueeze(0)
        label = self.image_df["label"][index]

        return img_tensor, label



In [1]:
class MonoLateFusModel(nn.Module):
    def __init__(self, text_model, img_model, tokenizer, text_dims, img_dims):
        super().__init__()
        self.tokenizer = tokenizer
        self.text_model = text_model
        self.img_model = img_model


        # architecture
        #ToxicBERT/HateBERT input
        self.textfc = nn.Linear(text_dims, 16)
        #ResNet input
        self.imgfc = nn.Linear(img_dims, 16)
        self.fusion = nn.Sequential(nn.Linear(32, 16), nn.Relu, nn.Linear(16, 2))

    def forward(self, tokenized_text, processed_image):
        #to be done
        txt = tokenized_text
        img = processed_image

        txt_proc = self.text_model()
        img_proc = self.img_model()
        combined_outputs = t.cat([txt_proc, img_proc], 1)
        outputs = self.fusion(combined_outputs)

        return outputs
    # def process_text(self, text):
    #     #take da text n tokenize that shit, then put it in da model
    #     model = self.text_model
    #     tokenizer = tr.AutoTokenizer.from_pretrained("unitary/toxic-bert")
    #     toktext = tokenizer(text)
    #     return toktext

    
    # def process_image(self, image_path):
        #idk resnet50?
    #     model = self.img_model
    #     transform_img = transforms.Compose([
    #                 transforms.Resize((224, 224)),
    #                 transforms.ToTensor(),
    #                 transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    #             ])
    #     img = Image.open(image_path).convert("RGB")
    #    img_mtrx = transform_img(img).unsqueeze(0)
    #     return img_mtrx
    
    def fine_tune_subpart(self, dataloader, input_type):
        model = self.text_model if input_type == "txt" else self.img_model
        model.train()

        tot_loss = 0
        correct_preds = 0


        for e in range(3):
            if input_type == "txt":
                for batch in dataloader:
                    toktext = batch["input_ids"]
                    mask = batch["attention_mask"]
                    labels = batch["labels"]
                    outputs = model(input_ids=toktext, attention_mask=mask, labels=labels)

                    # loss= nn.CrossEntropyLoss()
                    optimizer = optim.AdamW(model.parameters, lr= 5e-5)
                    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.9)

                    loss = outputs.loss
                    logits = outputs.logits

                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()
                    scheduler.step()
                    tot_loss += loss.item()

                    pred = t.softmax(logits, 1)

            elif input_type == "img":
                for images, labels in dataloader:

                    criterion = nn.CrossEntropyLoss()
                    optimizer = optim.Adam(model.fc.parameters(), lr=0.001)

                    optimizer.zero_grad()
                    outputs = model(images)
                    loss = criterion(outputs, labels)
                    loss.backward()
                    optimizer.step()
                    tot_loss += loss.item()
                    
                    _, pred = t.max(outputs.data, 1)
            
            else:
                print ("input_type must be either 'txt' or 'img'")
                
            correct_preds = t.sum(pred == labels)
            return "loss:", tot_loss/len(dataloader), "accuracy:", correct_preds/len(dataloader)











SyntaxError: positional argument follows keyword argument unpacking (4076218601.py, line 54)